# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [ ]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10) # 1이 spam 데이터이다

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [2]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], spam_data['target'], 
                                                    random_state=0)

### Question 1
What percentage of the documents in `spam_data` are spam?

*This function should return a float, the percent value (i.e. $ratio * 100$).*

In [3]:
def answer_one():
    return sum(spam_data.target == 1) / len(spam_data.target) * 100 #Your answer here

In [4]:
answer_one()

13.406317300789663

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

def answer_two():
    vect = CountVectorizer().fit(X_train)
    default = 0
    word = 'a'
    for i in vect.get_feature_names_out():
        if int(len(i)) > default:
            default = len(i)
            word = i
    
    return word#Your answer here

In [6]:
answer_two()

'com1win150ppmx3age16subscription'

### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [7]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def answer_three():
    vect = CountVectorizer().fit(X_train)
    X_train_transform = vect.transform(X_train)
    X_test_transform = vect.transform(X_test)

    naive_model = MultinomialNB(alpha=0.1)
    naive_model.fit(X_train_transform, y_train)
    
    return roc_auc_score(y_test, naive_model.predict(X_test_transform))#Your answer here

In [8]:
answer_three()

0.9720812182741116

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf?

Put these features in a two series where each series is sorted by tf-idf value and then alphabetically by feature name. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. 

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

def answer_four():
    model = TfidfVectorizer().fit(X_train)
    feature_names = np.array(model.get_feature_names())
    X_train_transform = model.transform(X_train)
    max_tf = X_train_transform.max(0).toarray()[0]
    max_tf_idx = max_tf.argsort()
    feature_name_sort = feature_names[max_tf_idx]
    
    final_data = max_tf[max_tf_idx]

    return feature_name_sort

answer_four()

c:\Users\user\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


array(['sympathetic', 'healer', 'aaniye', ..., 'home', 'havent',
       '146tf150p'], dtype='<U32')

In [10]:
answer_four()

array(['sympathetic', 'healer', 'aaniye', ..., 'home', 'havent',
       '146tf150p'], dtype='<U32')

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [11]:
def answer_five():
    model = TfidfVectorizer(min_df=3).fit(X_train)
    X_train_transform = model.transform(X_train)
    X_test_transform = model.transform(X_test)
    
    naive_model = MultinomialNB(alpha=0.1)
    naive_model.fit(X_train_transform, y_train)
    return roc_auc_score(y_test, naive_model.predict(X_test_transform))  #Your answer here

In [12]:
answer_five()

0.9416243654822335

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [19]:
def answer_six():
    avg_notspam = spam_data.query('target == 0')['text'].apply(len).values.mean()
    avg_spam = spam_data.query('target == 1')['text'].apply(len).values.mean()    
    
    return avg_notspam, avg_spam#Your answer here

In [20]:
answer_six()

(71.02362694300518, 138.8661311914324)

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [15]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [16]:
from sklearn.svm import SVC

def answer_seven():
    model = TfidfVectorizer(min_df=5).fit(X_train)
    X_train_transform = model.transform(X_train)
    X_test_transform = model.transform(X_test)    
    feature_add_train = X_train.apply(len).values
    feature_add_test = X_test.apply(len).values

    X_train_add = add_feature(X_train_transform, feature_to_add=feature_add_train)
    X_test_add = add_feature(X_test_transform, feature_to_add=feature_add_test)

    svc = SVC(C=10000)
    svc.fit(X_train_add, y_train)

    return roc_auc_score(y_test, svc.predict(X_test_add))#Your answer here

In [17]:
answer_seven()

0.9661689557407943

### Question 8

What is the average number of digits per document for not spam and spam documents?

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [19]:
import re

def digit(x):
    pure_digit = re.findall('[0-9]',x)
    return pure_digit
def answer_eight():
    digit_se1 = spam_data.query("target==1")["text"].apply(digit)
    digit_se0 = spam_data.query("target==0")["text"].apply(digit)
    digit_mean_se1 = digit_se1.apply(len).mean()
    digit_mean_se0 = digit_se0.apply(len).mean()
    return digit_mean_se0, digit_mean_se1

In [20]:
answer_eight()

(0.2992746113989637, 15.759036144578314)

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [21]:
from sklearn.linear_model import LogisticRegression

def answer_nine():
    tfid = TfidfVectorizer(min_df=5, ngram_range=(1,3))

    feature_len_train = X_train.apply(len).values
    feature_num_train = X_train.apply(digit).apply(len).values
    feature_len_test = X_test.apply(len).values
    feature_num_test = X_test.apply(digit).apply(len).values

    X_train_transform = tfid.fit_transform(X_train)
    X_test_transform = tfid.transform(X_test)

    X_train_add = add_feature(X_train_transform,feature_len_train)
    X_train_add2 = add_feature(X_train_add, feature_num_train)

    X_test_add = add_feature(X_test_transform,feature_len_test)
    X_test_add = add_feature(X_test_add,feature_num_test)

    logistic = LogisticRegression(C=100, max_iter=1000, n_jobs=-1)
    logistic.fit(X_train_add2, y_train)
    return roc_auc_score(y_test, logistic.predict(X_test_add))

In [22]:
answer_nine()

0.9678709064054463

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [51]:
def find_non_word(x):
    lst = re.findall("[\W]",x)
    return lst

def answer_ten():
    target0 = spam_data.query("target==0")["text"]
    target1 = spam_data.query("target==1")["text"]

    return target0.apply(find_non_word).apply(len).mean(), target1.apply(find_non_word).apply(len).mean()   

In [53]:
answer_ten()

(17.29181347150259, 29.041499330655956)

### Question 11

Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [59]:
def answer_eleven():
    count_vector = CountVectorizer(min_df=5, ngram_range=(2,5), analyzer='char_wb')
    
    feature_len_train = X_train.apply(len).values
    feature_num_train = X_train.apply(digit).apply(len).values
    feature_count_train = X_train.apply(find_non_word).apply(len).values
    
    feature_len_test = X_test.apply(len).values
    feature_num_test = X_test.apply(digit).apply(len).values
    feature_count_test = X_test.apply(find_non_word).apply(len).values
    
    
    X_train_transform = count_vector.fit_transform(X_train)
    X_test_transform = count_vector.transform(X_test)
    
    X_train_add = add_feature(X_train_transform,feature_len_train)
    X_train_add = add_feature(X_train_add, feature_num_train)
    X_train_add = add_feature(X_train_add, feature_count_train)
    
    X_test_add = add_feature(X_test_transform,feature_len_test)
    X_test_add = add_feature(X_test_add, feature_num_test)
    X_test_add = add_feature(X_test_add, feature_count_test)
    
    logistic_model = LogisticRegression(C=100)
    logistic_model.fit(X_train_add, y_train)
    
    score = roc_auc_score(y_test, logistic_model.predict(X_test_add))
    
    name = np.array(count_vector.get_feature_names() + ['length_of_doc', 'digit_count', 'non_word_char_count'])
    sorted_coef_index = logistic_model.coef_[0].argsort()
    smallest = name[sorted_coef_index[:10]]
    largest = name[sorted_coef_index[:-11:-1]]
    
    return score, smallest, largest

In [60]:
answer_eleven()

c:\Users\user\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\user\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


(0.9813973821367333,
 array(['..', '. ', ' i', ' go', ' y', '? ', 'pe', 'go', 'h ', 'ok'],
       dtype='<U19'),
 array(['digit_count', 'co', 'ww', 'ne', 'ar', 'ia', ' ch', 'mob', 'xt',
        'uk'], dtype='<U19'))